In [ ]:
import os
import time
import datetime

import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import DataLoader
import wandb

from config.config import load_config
from utils.utils import *
from datasets.transforms import build_unified_transforms

from datasets import get_dataset
from models import get_model

from utils.EarlyStopping import EarlyStopping
from utils.optimizer_factory import get_optimizer
from utils.scheduler_factory import get_scheduler
from trainer.train_loop import training_loop
from trainer.wandb_logger import WandbLogger

In [2]:
# 시드를 고정합니다.
SEED = 42
set_seed(SEED)

seed 고정 완료!


In [ ]:
cfg = load_config("config/main_config.yaml")
train_transform, val_transform = build_unified_transforms(cfg["transforms"]["train"]), build_unified_transforms(cfg["transforms"]["val"])

DatasetClass = get_dataset(cfg['DATASET'])
ModelClass = get_model(cfg['MODEL'])
cfg_scheduler = cfg["scheduler"]
cfg_optimizer = cfg["optimizer"]

In [4]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = './data'

# output config
output_root = './output'

# training config
num_workers = os.cpu_count() // 2
num_classes = 17
meta_df = pd.read_csv(f"{data_path}/meta_kr.csv")
class_names = meta_df["class_name"].tolist()

In [5]:
date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
filename = f"{cfg['MODEL']}_{date}"

# wandb
logger = WandbLogger(
    project_name="document-type-classification",
    run_name=filename,
    config=cfg,
    save_path=f"{output_root}/checkpoint.pth"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc
wandb: Currently logged in as: fkjy132 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


✅ 적용된 폰트: 'NanumGothic'
✅ PDF/PS 폰트 타입 42로 설정 완료 (한글 깨짐 방지)


In [6]:
# Dataset 정의
train_dataset = DatasetClass(
    f"{data_path}/train_valid_set/train-label-fix-v1.csv",
    f"{data_path}/train/",
    transform=train_transform
)
val_dataset = DatasetClass(
    f"{data_path}/train_valid_set/val-v1.csv",
    f"{data_path}/train/",
    transform=val_transform
)
print(len(train_dataset), len(val_dataset))

1255 315


In [7]:
# DataLoader 정의
train_loader = DataLoader(
    train_dataset,
    batch_size=cfg["BATCH_SIZE"],
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False,
    persistent_workers=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=cfg["BATCH_SIZE"],
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False,
    persistent_workers=True
)

In [ ]:
# load model
model: nn.Module = ModelClass(num_classes=num_classes).to(device)

# 1. 모델의 모든 파라미터를 우선 동결(freeze)합니다.
for param in model.parameters():
    param.requires_grad = False

# 2. 특징 추출기(backbone)의 마지막 2개 블록의 동결을 해제(unfreeze)합니다.
# efficientnet_b3는 7개의 블록(0~6)을 가집니다.
num_blocks_to_unfreeze = 2
for i in range(num_blocks_to_unfreeze):
    for param in model.backbone.blocks[-(i+1)].parameters():
        param.requires_grad = True

# 3. 분류기(head)의 동결을 해제합니다.
for param in model.head.parameters():
    param.requires_grad = True
    
# 4. 학습시킬 파라미터만 필터링하여 옵티마이저에 전달합니다.
# requires_grad=True인 파라미터만 업데이트됩니다.
params_to_update = filter(lambda p: p.requires_grad, model.parameters())


early_stopping = EarlyStopping(patience=cfg["patience"], delta=cfg["delta"], verbose=True, save_path=f'{output_root}/checkpoint.pth')

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = get_optimizer(cfg_optimizer["name"], model.parameters(), cfg_optimizer["params"])
# optimizer = get_optimizer(cfg_optimizer["name"], params_to_update, cfg_optimizer["params"])

# 스케쥴러
Scheduler = get_scheduler(cfg_scheduler["name"], optimizer, cfg_scheduler['params'])

model, valid_max_accuracy = training_loop(model, train_loader, val_loader, train_dataset, val_dataset, criterion, optimizer, device, cfg["EPOCHS"], early_stopping, logger, class_names, Scheduler)

Epoch [1/100], Valid Loss: 1.5527: 100%|██████████| 20/20 [00:59<00:00,  2.99s/it]


  ✅ Validation loss improved. Saving model...
Epoch [1/100]
Train Loss: 2.2065, Train Accuracy: 0.4375, Train f1: 0.3735941857291823
Valid Loss: 1.7674, Valid Accuracy: 0.4889, Valid f1: 0.43185355122945435
Current LR: 0.00009758


Epoch [2/100], Valid Loss: 1.8165: 100%|██████████| 20/20 [00:55<00:00,  2.76s/it]


  ✅ Validation loss improved. Saving model...
Epoch [2/100]
Train Loss: 1.1019, Train Accuracy: 0.6821, Train f1: 0.6151495665693703
Valid Loss: 1.4045, Valid Accuracy: 0.5619, Valid f1: 0.4849615010924865
Current LR: 0.00009055


Epoch [3/100], Valid Loss: 0.6819: 100%|██████████| 20/20 [00:57<00:00,  2.86s/it]


...

Epoch [30/100], Valid Loss: 0.0440: 100%|██████████| 20/20 [00:58<00:00,  2.94s/it]


  ↪️ No improvement. EarlyStopping counter: 9/10
Epoch [30/100]
Train Loss: 0.0960, Train Accuracy: 0.9665, Train f1: 0.9634987659362285
Valid Loss: 0.5600, Valid Accuracy: 0.8095, Valid f1: 0.7893058772517595
Current LR: 0.00010000


Epoch [31/100], Valid Loss: 0.3688: 100%|██████████| 20/20 [00:54<00:00,  2.73s/it]


  ↪️ No improvement. EarlyStopping counter: 10/10
Epoch [31/100]
Train Loss: 0.1224, Train Accuracy: 0.9610, Train f1: 0.9594282446493352
Valid Loss: 0.6159, Valid Accuracy: 0.8159, Valid f1: 0.7906598194029503
Current LR: 0.00009985
🛑 Early stopping at epoch 31


lr,█▇▇▆▄▃▂▂▁████▇▇▇▆▆▅▄▄▃▃▂▂▂▁▁▁██
train/acc,▁▄▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇█████████████
train/f1,▁▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇█▇█████████████
train/loss,█▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/acc,▁▃▅▅▆▆▆▆▇▆▇▇▆▇▇▇▇▇▇███▇▇▇▇▇████
val/f1,▁▂▄▅▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇███▇▇▇█▇████
val/loss,█▆▄▃▃▂▃▃▂▃▂▂▂▂▁▂▂▂▁▁▁▁▂▂▂▂▂▁▁▁▁
lr,0.0001
train/acc,0.96096
train/f1,0.95943
train/loss,0.12236


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [9]:
import os

import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader

from config.config import load_config
from models import get_model
from datasets import get_dataset
from datasets.transforms import build_unified_transforms

In [10]:
cfg = load_config("config/main_config.yaml")
ModelClass = get_model(cfg['MODEL'])
DatasetClass = get_dataset(cfg['DATASET'])
num_classes = 17
num_workers = os.cpu_count() // 2
output_root = './output'
data_path = './data'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

test_transform = build_unified_transforms(cfg["transforms"]["test"])

tst_dataset = DatasetClass(
    f"{data_path}/sample_submission.csv",
    f"{data_path}/test/",
    transform=test_transform
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size=cfg["BATCH_SIZE"],
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True
)

In [11]:
preds_list = []
model = ModelClass(num_classes).to(device)
model.load_state_dict(torch.load(f"{output_root}/checkpoint.pth", map_location="cpu"))

model.eval()
for image, _, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 197/197 [00:22<00:00,  8.67it/s]


In [12]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [13]:
sample_submission_df = pd.read_csv(f"{data_path}/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [14]:
pred_df.to_csv("pred.csv", index=False)

In [ ]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,6
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,6
4,00901f504008d884.jpg,2


: 